In [6]:
import sys
print(sys.version)

3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0]


In [7]:
import torch
print(torch.__version__)

2.1.1+cu118


In [61]:
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv
import torch.nn.functional as F
import torch.nn as nn

In [68]:
from tqdm import tqdm
import numpy as np

In [8]:
device = torch.device("cuda")

In [11]:
cora_dataset = Planetoid('/tmp/cora', 'cora')

Processing...
Done!


In [19]:
cora_data = cora_dataset[0]

In [27]:
# For debug use only
num_nodes = cora_data.num_nodes
print('cora has {} nodes'.format(num_nodes))

num_edges = cora_data.num_edges
print('cora has {} edges'.format(num_edges))

cora has 2708 nodes
cora has 10556 edges


In [28]:
# For debug use only
print(cora_data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [30]:
cora_x_train = cora_data.x[cora_data.train_mask]
cora_x_val = cora_data.x[cora_data.val_mask]
cora_x_test = cora_data.x[cora_data.test_mask]

print("number of nodes in cora train set,", cora_x_train.shape[0])
print("number of nodes in cora val set,", cora_x_val.shape[0])
print("number of nodes in cora test set,", cora_x_test.shape[0])

number of nodes in cora train set, 140
number of nodes in cora val set, 500
number of nodes in cora test set, 1000


In [38]:
# For debug use only
print(cora_data.y)
print(cora_data.y.shape)
s = set()
histogram = np.zeros(7)
for label in cora_data.y:
    s.add(label.item())
    histogram[label.item()]+=1
print(s)
print(histogram)

tensor([3, 4, 4,  ..., 3, 3, 3])
torch.Size([2708])
{0, 1, 2, 3, 4, 5, 6}
[351. 217. 418. 818. 426. 298. 180.]


In [53]:
# For debug use only
print(cora_data.x.shape)
print(cora_data.x[170:180])
print(cora_data.num_features)
print(cora_data.num_nodes)
print(cora_data.num_node_types)
print(type(cora_data))

torch.Size([2708, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
1433
2708
1
<class 'torch_geometric.data.data.Data'>


In [62]:
# Define the GAT model
class GAT(torch.nn.Module):
    # hidden channels will be the embedding dimension for each attention head
    # after applying the first GAT layer.
    def __init__(self, in_channels, hidden_channels, 
                 num_heads, dropout_rate, num_classes):
        super().__init__()
        
        self.dropout_rate = dropout_rate
        
        self.hidden_channels = hidden_channels
        self.num_heads = num_heads
        
        self.conv1 = GATConv(in_channels, hidden_channels, heads=num_heads, 
                                dropout=dropout_rate)
        self.conv2 = GATConv(hidden_channels*num_heads, num_classes, 
                                dropout=dropout_rate, concat=False)

    def forward(self, x, edge_index):
        out = F.dropout(x, p=self.dropout_rate, training=self.training)
        
        out = self.conv1(out, edge_index)
        assert out.shape[-1] == self.hidden_channels * self.num_heads
        
        out = F.elu(out)
        out = F.dropout(out, p=self.dropout_rate, training=self.training)
        
        out = self.conv2(out, edge_index)
        return out

In [63]:
num_heads = 8
dropout_rate = 0.4
emb_dim1 = 8
lr = 0.005

cora_num_classes = len(cora_data.y.unique())
assert cora_num_classes == 7

cora_model = GAT(cora_data.num_features, emb_dim1, num_heads, dropout_rate, 
            cora_num_classes).to(device)
optimizer = torch.optim.Adam(cora_model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

In [64]:
def train(model, data, optimizer, loss_fn):
    model.train()
    optimizer.zero_grad()
    
    pred = model(data.x, data.edge_index)
    loss = loss_fn(pred[data.train_mask], data.y[data.train_mask])
    
    loss.backward()
    optimizer.step()
    
    return loss

In [71]:
@torch.no_grad()
def evaluate(model, data, test_mask):
    accuracy_list = [0.0, 0.0]
    # loss_list = [0.0, 0.0]
    model.eval()

    logits = model(data.x, data.edge_index)
    pred = logits.argmax(dim=-1)
    
    for i, mask in enumerate([data.train_mask, test_mask]):
        accuracy_list[i] = pred[mask].eq(data.y[mask]).float().mean().item()
        # loss_list[i] = loss_fn(logits[mask], data.y[mask]).item()

    return accuracy_list

In [73]:
num_epochs = 200
cora_data_gpu = cora_data.to(device)
# optimizer_gpu = optimizer.to(device)
# loss_fun_gpu = loss_fun.to(device)
for epoch in tqdm(range(num_epochs), desc="Training Epochs"):
    loss = train(cora_model, cora_data, optimizer, loss_fn)
    train_acc, val_acc = evaluate(cora_model, cora_data, cora_data.val_mask)
    print(f'Epoch: {epoch+1}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')

Training Epochs:  16%|█▌        | 31/200 [00:00<00:01, 154.17it/s]

Epoch: 1, Loss: 1.5795, Train Acc: 0.9286, Val Acc: 0.6640
Epoch: 2, Loss: 1.4267, Train Acc: 0.9429, Val Acc: 0.7180
Epoch: 3, Loss: 1.3247, Train Acc: 0.9429, Val Acc: 0.7420
Epoch: 4, Loss: 1.1349, Train Acc: 0.9500, Val Acc: 0.7660
Epoch: 5, Loss: 0.9936, Train Acc: 0.9643, Val Acc: 0.7800
Epoch: 6, Loss: 0.8978, Train Acc: 0.9714, Val Acc: 0.7880
Epoch: 7, Loss: 0.8970, Train Acc: 0.9714, Val Acc: 0.7880
Epoch: 8, Loss: 0.7726, Train Acc: 0.9714, Val Acc: 0.7900
Epoch: 9, Loss: 0.6817, Train Acc: 0.9857, Val Acc: 0.7860
Epoch: 10, Loss: 0.6420, Train Acc: 0.9929, Val Acc: 0.7860
Epoch: 11, Loss: 0.6064, Train Acc: 0.9929, Val Acc: 0.7820
Epoch: 12, Loss: 0.5011, Train Acc: 0.9929, Val Acc: 0.7820
Epoch: 13, Loss: 0.4454, Train Acc: 0.9929, Val Acc: 0.7780
Epoch: 14, Loss: 0.4813, Train Acc: 0.9929, Val Acc: 0.7800
Epoch: 15, Loss: 0.3980, Train Acc: 0.9929, Val Acc: 0.7760
Epoch: 16, Loss: 0.4239, Train Acc: 0.9929, Val Acc: 0.7780
Epoch: 17, Loss: 0.4009, Train Acc: 0.9929, Val A

Training Epochs:  34%|███▍      | 69/200 [00:00<00:00, 177.22it/s]

Epoch: 38, Loss: 0.1924, Train Acc: 1.0000, Val Acc: 0.7680
Epoch: 39, Loss: 0.1638, Train Acc: 1.0000, Val Acc: 0.7680
Epoch: 40, Loss: 0.1678, Train Acc: 1.0000, Val Acc: 0.7680
Epoch: 41, Loss: 0.1536, Train Acc: 1.0000, Val Acc: 0.7660
Epoch: 42, Loss: 0.2103, Train Acc: 1.0000, Val Acc: 0.7620
Epoch: 43, Loss: 0.1630, Train Acc: 1.0000, Val Acc: 0.7620
Epoch: 44, Loss: 0.1622, Train Acc: 1.0000, Val Acc: 0.7620
Epoch: 45, Loss: 0.1434, Train Acc: 1.0000, Val Acc: 0.7620
Epoch: 46, Loss: 0.1332, Train Acc: 1.0000, Val Acc: 0.7620
Epoch: 47, Loss: 0.1528, Train Acc: 1.0000, Val Acc: 0.7620
Epoch: 48, Loss: 0.1837, Train Acc: 1.0000, Val Acc: 0.7640
Epoch: 49, Loss: 0.1608, Train Acc: 1.0000, Val Acc: 0.7640
Epoch: 50, Loss: 0.1248, Train Acc: 1.0000, Val Acc: 0.7660
Epoch: 51, Loss: 0.1462, Train Acc: 1.0000, Val Acc: 0.7640
Epoch: 52, Loss: 0.1173, Train Acc: 1.0000, Val Acc: 0.7620
Epoch: 53, Loss: 0.1142, Train Acc: 1.0000, Val Acc: 0.7620
Epoch: 54, Loss: 0.1638, Train Acc: 1.00

Training Epochs:  54%|█████▎    | 107/200 [00:00<00:00, 183.96it/s]

...

Epoch: 118, Loss: 0.1008, Train Acc: 1.0000, Val Acc: 0.7620
Epoch: 119, Loss: 0.1190, Train Acc: 1.0000, Val Acc: 0.7620
Epoch: 120, Loss: 0.0962, Train Acc: 1.0000, Val Acc: 0.7620
Epoch: 121, Loss: 0.0729, Train Acc: 1.0000, Val Acc: 0.7620
Epoch: 122, Loss: 0.1364, Train Acc: 1.0000, Val Acc: 0.7640
Epoch: 123, Loss: 0.0653, Train Acc: 1.0000, Val Acc: 0.7640
Epoch: 124, Loss: 0.1157, Train Acc: 1.0000, Val Acc: 0.7620
Epoch: 125, Loss: 0.1754, Train Acc: 1.0000, Val Acc: 0.7580
Epoch: 126, Loss: 0.1388, Train Acc: 1.0000, Val Acc: 0.7540
Epoch: 127, Loss: 0.0821, Train Acc: 1.0000, Val Acc: 0.7560
Epoch: 128, Loss: 0.0893, Train Acc: 1.0000, Val Acc: 0.7560
Epoch: 129, Loss: 0.0774, Train Acc: 1.0000, Val Acc: 0.7560
Epoch: 130, Loss: 0.0927, Train Acc: 1.0000, Val Acc: 0.7560
Epoch: 131, Loss: 0.0618, Train Acc: 1.0000, Val Acc: 0.7560
Epoch: 132, Loss: 0.1338, Train Acc: 1.0000, Val Acc: 0.7540
Epoch: 133, Loss: 0.1192, Train Acc: 1.0000, Val Acc: 0.7540
Epoch: 134, Loss: 0.0600

Training Epochs:  92%|█████████▎| 185/200 [00:01<00:00, 189.26it/s]

Epoch: 158, Loss: 0.1217, Train Acc: 1.0000, Val Acc: 0.7520
Epoch: 159, Loss: 0.1449, Train Acc: 1.0000, Val Acc: 0.7500
Epoch: 160, Loss: 0.0738, Train Acc: 1.0000, Val Acc: 0.7500
Epoch: 161, Loss: 0.0718, Train Acc: 1.0000, Val Acc: 0.7540
Epoch: 162, Loss: 0.1437, Train Acc: 1.0000, Val Acc: 0.7500
Epoch: 163, Loss: 0.0576, Train Acc: 1.0000, Val Acc: 0.7500
Epoch: 164, Loss: 0.1202, Train Acc: 1.0000, Val Acc: 0.7520
Epoch: 165, Loss: 0.0652, Train Acc: 1.0000, Val Acc: 0.7540
Epoch: 166, Loss: 0.1605, Train Acc: 1.0000, Val Acc: 0.7540
Epoch: 167, Loss: 0.1145, Train Acc: 1.0000, Val Acc: 0.7520
Epoch: 168, Loss: 0.0880, Train Acc: 1.0000, Val Acc: 0.7560
Epoch: 169, Loss: 0.1450, Train Acc: 1.0000, Val Acc: 0.7580
Epoch: 170, Loss: 0.1157, Train Acc: 1.0000, Val Acc: 0.7580
Epoch: 171, Loss: 0.0883, Train Acc: 1.0000, Val Acc: 0.7580
Epoch: 172, Loss: 0.0718, Train Acc: 1.0000, Val Acc: 0.7580
Epoch: 173, Loss: 0.0974, Train Acc: 1.0000, Val Acc: 0.7580
Epoch: 174, Loss: 0.0970

Training Epochs: 100%|██████████| 200/200 [00:01<00:00, 181.86it/s]

Epoch: 198, Loss: 0.0949, Train Acc: 1.0000, Val Acc: 0.7600
Epoch: 199, Loss: 0.0970, Train Acc: 1.0000, Val Acc: 0.7620
Epoch: 200, Loss: 0.0535, Train Acc: 1.0000, Val Acc: 0.7620
